In [1]:
%matplotlib inline

import os
import itertools

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy

from skimage import color
from skimage.transform import hough_circle, hough_circle_peaks, rotate
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.filters import threshold_otsu

import tifffile.tifffile

############################
####### PARAMETERS #########
############################

path = 'Path/To/Data/' #This folder has folders within it, each with a stack called stackname
stackname = 'cc.tif'
writepath = 'Path/To/Data/output'

pixelsize_xy = 0.466/5    #there is a five pixel interpolation, units of nm/pixel
pixelsize_z = 0.466    #slice spacing, units of nm/pixel

#Main variables for setting search space
maxskew = 4 #degrees. minskew = -maxskew
skew_interval = 2 #This should be small, like 0.25
minfoldsym = 9 #pf number
maxfoldsym = 15

############################
####### ADVANCED ###########
############################

foldsym_interval = 1 #LEAVE THIS AT 1
START = 1 #Which folder to start at, good for when code crashes. Beginning is 1

#Sample spacing for plot profile
spacing = 15000
num_circles = 100

############################
######### START! ###########
############################

#Create list of skews and folds from values above
skewlist = np.arange(-maxskew, maxskew + skew_interval, skew_interval)
foldslist = np.arange(minfoldsym, maxfoldsym + foldsym_interval, foldsym_interval)
totskews = len(skewlist)
totfolds = len(foldslist)

#Initialize arrays for results
numslices_lst = []
filelist = []
skew_atmax = []
fold_atmax = []
diam_atmax = []
amplitudes = np.zeros((totskews, totfolds))
diameters = np.zeros((totskews, totfolds))
noises = np.zeros((totskews, totfolds))

#Total for print
total = totskews*totfolds

#Iterater for folder search
increment = 1

#Function to convert pf skew to inter-slice rotation angle
def interslice(skew):
    return(pixelsize_z*np.degrees(np.tan(np.radians(skew)))/10)

for root, dirs, files in os.walk(path):
    for name in files:
        if name == stackname:
            if increment >= START:
                
                #Iterator for condition number
                n = 1

                filepath = os.path.join(root, name)
                im = tifffile.imread(filepath)
                
                imagenum = str(increment)
                #imagenum = root[-5:-3] + "_" + root[-2:]
                filelist.append(imagenum)

                stack = []

                for a in im:
                    stack.append(a)

                #Convert to array and invert intensities
                stack = ~np.array(stack)

                #some simple variables
                numslices = stack.shape[0]
                numslices_lst.append(stack.shape[0])
                width = stack.shape[1]

                ##########################
                ######### SKEW ###########
                ##########################

                #Iterate over all skew angles
                for s, skew in enumerate(skewlist):

                    #Initialize stack to store rotated images
                    skew_stack = []
                    
                    #Convert pf skew to slice rotation
                    skew = interslice(skew)

                    #Iterate over all slices in stack and apply skew to each
                    for i in range(0, numslices, 1):

                        #Determine how much to rotate based on skew angle and slice number
                        sliceskew = i * skew

                        #Apply skew rotation to slice
                        skewed_image = rotate(stack[i], sliceskew)

                        #Rotation creates dark areas where pixels are absent, change those to median value
                        #THIS FUCKS THIS UP, LEFT OUT AND TOLERATED
                        #skewed_image[skewed_image == 0] = ms

                        #Add rotated image to stack of skewed images
                        skew_stack.append(skewed_image)

                    #Take the projection by summing all images
                    projection = np.sum(skew_stack, axis=0)

                    ##########################
                    ####### ROTATION #########
                    ##########################

                    # Compute the median to replace background values after rotation (see below)
                    mf = np.median(projection)

                    #Iterate over all fold-symmetries
                    for f, foldsym in enumerate(foldslist):

                        print("Working on " + imagenum + ": " + str(n)+ "/" + str(total), end='\r', flush=True)

                        #Calculate angle to rotate by
                        rotangle = 360/foldsym

                        #Calculate the total number of rotations required
                        totfolds = int(foldsym)

                        #Initialize stack to store rotated images
                        rotavg_stack = []

                        #Create stack of rotations
                        for i in range(0, totfolds, 1):

                            #Determine how much to rotate based on iteration nu, axis=0mber
                            slice_rotangle = rotangle * i            

                            #Apply rotation
                            rotated_image = rotate(projection, slice_rotangle)            

                            #Rotation creates dark areas where pixels are absent, change those to median value
                            rotated_image[rotated_image == 0] = mf

                            #Add rotated image to stack of rotated images
                            rotavg_stack.append(rotated_image)

                        #Compute the rotational average by taking the average of all images   
                        rotavg = np.mean(rotavg_stack, axis=0)

                        ##########################
                        ####### THRESHOLD ########
                        ##########################

                        thresh = threshold_otsu(rotavg)
                        binary = rotavg > thresh

                        ##########################
                        ####### HOUGH-LINE #######
                        ##########################

                        # Create Houg90magesum + h circle with radii between 50 and 200.
                        hough_radii = np.linspace(50, 200, 50)
                        hough_res = hough_circle(binary, hough_radii)

                        # Select 10 circles
                        accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=10, min_xdistance=0, min_ydistance=0)

                        # Get the circle with the smallest radius
                        min_circle_index = np.argmin(radii)
                        min_circle_radius = radii[min_circle_index]
                        min_circle_x = cx[min_circle_index]
                        min_circle_y = cy[min_circle_index]

                        # Get the circle with the biggest radius
                        max_circle_index = np.argmax(radii)
                        max_circle_radius = radii[max_circle_index]
                        max_circle_x = cx[max_circle_index]
                        max_circle_y = cy[max_circle_index]

                        # Compute the average circle
                        circle_x = (min_circle_x + max_circle_x) / 2
                        circle_y = (min_circle_y + max_circle_y) / 2
                        circle_radius = (min_circle_radius + max_circle_radius) / 2
                        circle = np.array([circle_x, circle_y, circle_radius])

                        ##########################
                        ########## RADII #########
                        ##########################

                        #OUTER
                        for i in range(60):

                            new_radius = circle_radius + i

                            # x = circle_x + radius * sinus(angle)
                            x_points = np.array([circle_x + new_radius * np.sin(np.deg2rad(angle)) for angle in np.linspace(0, 360, spacing)])

                            # y = circle_y + radius * cosinus(angle)
                            y_points = np.array([circle_y + new_radius * np.cos(np.deg2rad(angle)) for angle in np.linspace(0, 360, spacing)])

                            # Get the intensity profile by interpolation
                            profile_bin = scipy.ndimage.map_coordinates(binary, np.vstack((x_points, y_points)))

                            # Record diameter once mean falls below 0.1
                            if np.mean(profile_bin) < 0.1:

                                outer_radius = new_radius
                                break

                        truediam = outer_radius * 2 * pixelsize_xy

                        #INNER
                        for i in range(60):

                            new_radius = circle_radius - i

                            # x = circle_x + radius * sinus(angle)
                            x_points = np.array([circle_x + new_radius * np.sin(np.deg2rad(angle)) for angle in np.linspace(0, 360, spacing)])

                            # y = circle_y + radius * cosinus(angle)
                            y_points = np.array([circle_y + new_radius * np.cos(np.deg2rad(angle)) for angle in np.linspace(0, 360, spacing)])

                            # Get the intensity profile by interpolation
                            profile_bin = scipy.ndimage.map_coordinates(binary, np.vstack((x_points, y_points)))

                            # Record diameter once mean falls below 0.1
                            if np.mean(profile_bin) < 0.1:

                                inner_radius = new_radius
                                break

                        ##########################
                        ######## PROFILE #########
                        ##########################

                        profiles = []

                        #Create a 3nm thick line to make a series of plot profiles out of
                        center_radius = (outer_radius + inner_radius)/2
                        inner_prof_radius = center_radius - 1.5/pixelsize_xy 
                        outer_prof_radius = center_radius + 2/pixelsize_xy 

                        for radius in np.linspace(inner_prof_radius, outer_prof_radius, num_circles):

                            # x = circle_x + radius * sinus(angle)
                            x_points = np.array([circle_x + radius * np.sin(np.deg2rad(angle)) for angle in np.linspace(0, 360, spacing)])

                            # y = circle_y + radius * cosinus(angle)
                            y_points = np.array([circle_y + radius * np.cos(np.deg2rad(angle)) for angle in np.linspace(0, 360, spacing)])

                            # Get the intensity profile by interpolation
                            profile = scipy.ndimage.map_coordinates(rotavg, np.vstack((x_points, y_points)))

                            profiles.append(profile)

                        profiles = np.array(profiles)
                        profile = np.mean(profiles, axis=0)
                        profile = profile - np.mean(profile)

                        ##########################
                        ######## FOURIER #########
                        ##########################

                        # Compute the FFT
                        ft = np.fft.rfft(profile * np.hanning(len(profile)))
                        ft = np.abs(ft)
                        ft = ft[:int(len(profile)/2)]

                        freqs = np.fft.fftfreq(len(profile), d=1)
                        freqs = np.around(freqs[0:len(ft)],4)
                        
                        amplitude_sym = []
                        freq_list = range(foldsym, foldsym*11, foldsym)
                        [amplitude_sym.append(ft[fr]) for fr in freq_list]

                        noise = np.sum(amplitude_sym[1:])
                        
                        ##########################
                        ########## SAVE ##########
                        ##########################

                        amplitudes[s,f] = amplitude_sym[0]
                        diameters[s,f] = truediam
                        noises[s,f] = noise

                        #Iterater for condition number print
                        n = n + 1

                #END

                # Convert to DataFrame
                amplitudes_df = pd.DataFrame(amplitudes.T)
                diameters_df = pd.DataFrame(diameters.T)
                noises_df = pd.DataFrame(noises.T)

                diameters_df.columns = np.around(skewlist,2)
                diameters_df.index = np.around(foldslist,1)
                amplitudes_df.columns = np.around(skewlist,2)
                amplitudes_df.index = np.around(foldslist,1)
                noises_df.columns = np.around(skewlist,2)
                noises_df.index = np.around(foldslist,1)

                # Save
                amplitudes_df.to_csv(os.path.join(writepath, imagenum + "_amplitudes.csv"))
                amplitudes_df.head()

                diameters_df.to_csv(os.path.join(writepath, imagenum + "_diameters.csv"))
                diameters_df.head()

                noises_df.to_csv(os.path.join(writepath, imagenum + "_noises.csv"))
                noises_df.head()           

                print("Done " + imagenum + "                     ")
            
            increment += 1


C:\ProgramData\Anaconda3\lib\site-packages\tifffile\tifffile.py:306: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "


Done 1                     
Done 2                     
Done 3                     
Done 4                     
Done 5                     
